# Fortegra Data Challenge 

A new potential partner is coming to us with a potential portfolio for us to work with. The portfolio is insurance for irrigation sprinklers, a type of farming equipment. They have been running this program for about 5 years and are looking for a new partner to work with them. We have an opportunity to analyze their data and recommend underwriting changes to improve the performance of their portfolio. We are targeting a loss rate of 60% or lower for any portfolio we work with. Loss ratio is defined as sum of total claims paid / sum of total premium earned, over a given time period.
There are 2 datasets attached: claims.csv has one row per claim filed and premiums.csv has one row per piece of equipment.
1. Clean the data how you see fit. State your assumptions. Assume that a cleaned subset of the data is better than data which is skewed, biased, or incorrect. Include conceptual validations/cleaning.
2. Calculate loss ratio by state, county, customer’s number of previous claims, deductible amount, and equipment year. Explain your results and note anything you found interesting or important.
3. Perform other pieces of EDA/analysis you think is interesting. Since time is limited, you can limit to 1-2 analyses, then list other analysis you think would be interesting to look at if you had more time.
4. Based on the analysis you’ve done, what are some recommended changes to the underwriting of this portfolio?


In [12]:
# Generic Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
claims_data = '../data/claims.csv'
premiums_data = '../data/premiums.csv'

In [8]:
claims_df = pd.read_csv(claims_data)
premiums_df = pd.read_csv(premiums_data)

# Data Sanity Checks
## Claims Dataframe

#### Check for null or NaN values

In [11]:
claims_df.isna().sum()

PolicyNumber            0
CustomerId              0
DateOfLoss              0
ClaimReceivedDate       2
ClaimCause              0
TotalPaidToDate         0
Deductible              0
County                 31
State                  31
EquipmentYear          33
PolicyEffectiveDate     0
dtype: int64

In [18]:
claims_df[claims_df.isna().any(axis = 1)].head(10)

,PolicyNumber,CustomerId,DateOfLoss,ClaimReceivedDate,ClaimCause,TotalPaidToDate,Deductible,County,State,EquipmentYear,PolicyEffectiveDate
44,P-4875-2022,4254,2022-09-05,2022-10-17,Claim Denied,626.50,2500,NaN,NaN,NaN,2022-05-02
140,P-1206-2022,4706,2022-07-05,2022-07-07,Claim Denied,0.00,2500,NaN,NaN,NaN,2022-07-05
335,P-538-2022,3691,2022-05-05,2022-05-09,Claim Withdrawn,0.00,1000,NaN,NaN,NaN,2021-06-01
408,P-2668-2022,2170,2022-04-22,2022-05-11,Claim Denied,0.00,1000,NaN,NaN,NaN,2021-07-30
441,P-4732-2022,510,2022-02-23,2022-05-18,Claim Denied,0.00,1000,NaN,NaN,NaN,2022-02-23
592,P-38-2021,2340,2021-08-19,2021-12-10,Collision-Irr,19224.88,2500,NaN,NaN,NaN,2021-05-15
756,P-5308-2021,967,2021-06-11,2021-06-21,Collision-Irr,50000.00,2500,NaN,NaN,NaN,2020-08-07
813,P-5057-2021,5192,2021-04-01,2021-04-15,Claim Denied,0.00,1000,NaN,NaN,NaN,2020-07-22
827,P-2486-2021,4191,2021-03-15,2021-03-18,Claim Withdrawn,0.00,5000,NaN,NaN,NaN,2020-04-01
843,P-3871-2021,4499,2021-02-22,2021-03-10,Claim Withdrawn,0.00,1000,NaN,NaN,NaN,2021-02-03


#### - It appears that missing information may be linked to "Claim Withdrawn" and "Claim Denied" status in the ClaimCause field.
#### - Additionally, County, State, and EquipmentYear appear to usually be missing concurrently.
#### - The ClaimRecievedDate is missing twice; this may be due to some clerical error.

# Data Sanity Checks
## Premiums Dataframe

#### Check for null or NaN values

In [15]:
premiums_df.isna().sum()

PolicyNumber             0
CustomerId               0
Loc                      0
PolicyEffectiveDate      0
PolicyExpirationDate     0
PolicyPremium            0
LocationPremium          0
Deductible               0
LocValue                 0
County                   0
State                    0
Equip Year              83
Equip Value             83
dtype: int64

In [19]:
premiums_df[premiums_df.isna().any(axis = 1)].head(10)

,PolicyNumber,CustomerId,Loc,PolicyEffectiveDate,PolicyExpirationDate,PolicyPremium,LocationPremium,Deductible,LocValue,County,State,Equip Year,Equip Value
5359,P-1031-2022,1278,5,2022-06-01,2023-06-01,40634,2209.107960,1000,163935.90,Edwards,KS,NaN,NaN
5378,P-1031-2022,1278,14,2022-06-01,2023-06-01,40634,2309.700103,1000,147693.00,Edwards,KS,NaN,NaN
8622,P-1860-2019,4671,2,2019-05-06,2020-05-06,2248,358.949932,1000,22954.25,Crosby,TX,NaN,NaN
8624,P-1860-2019,4671,3,2019-05-06,2020-05-06,2248,154.125395,1000,9869.25,Hale,TX,NaN,NaN
8627,P-1860-2020,4671,2,2020-05-06,2021-05-06,2248,367.000000,1000,22954.25,Crosby,TX,NaN,NaN
8629,P-1860-2020,4671,3,2020-05-06,2021-05-06,2248,158.000000,1000,9869.25,Hale,TX,NaN,NaN
9279,P-1042-2018,3246,5,2018-03-26,2019-03-26,5392,557.000000,1000,33743.59,Clay,SD,NaN,NaN
14549,P-3697-2018,1589,6,2018-09-02,2019-09-02,8737,1285.000000,1000,80341.18,Tom Green,TX,NaN,NaN
14558,P-3697-2019,1589,6,2019-09-02,2020-09-02,12615,1380.067683,1000,80341.18,Tom Green,TX,NaN,NaN
14570,P-3697-2020,1589,6,2020-09-02,2021-09-02,12615,1285.000000,1000,80341.18,Tom Green,TX,NaN,NaN


#### - Equipment Year and Equipment Value are typically missing in concurrently.